In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive/hw2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/hw2


In [2]:
!ls

d2v_default.p	      download_ap.ipynb    __pycache__	     trec.py
datasets	      environment.yml	   read_ap.py	     WINDOWS.md
Doc2vec_default.json  hw2_description.pdf  requirements.txt
Dov2Vec2.ipynb	      processed_docs.pkl   tf_idf.py


In [6]:
! pip install pytrec-eval

In [3]:
import os
import gensim
import csv
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
# import trec
import pytrec_eval
import json


import read_ap
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pickle
def rank(query, model):
      inferred_vector = model.infer_vector(query, 10000)
      sims = model.docvecs.most_similar([inferred_vector], topn = len(model.docvecs))
      return sims
def print_similar(query, model, traincorpus):
  query = read_ap.process_text(query)
  sims = rank(query, model)
  # print('Document ({}): «{}»\n'.format(doc_id, ' '.join(traincorpus[doc_id].words)))
  print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
  for label, index in [('1', 0), ('2', 1), ('3', 2), ('4', 3), ('5', 4), ('6', 5), ('7', 6), ('8', 7), ('9', 8), ('10', 9)]:
      print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(docs_by_id[sims[index][0]])))

def main():

    qrels, queries = read_ap.read_qrels()
    docs_by_id = read_ap.get_processed_docs()

    traincorpus = [TaggedDocument(j, [i]) for i, j in docs_by_id.items()]

    # model = gensim.models.doc2vec.Doc2Vec(min_count=50, epochs=5)
    # model.build_vocab(traincorpus)
    # model.train(traincorpus, total_examples=model.corpus_count, epochs=model.epochs)
    model = pickle.load(open( "d2v_default.p", "rb" ))


    overall_ser = {}

    print("Running Doc2Vec Benchmark")
    # collect results
    for qid in tqdm(qrels): 
        query_text = queries[qid]
        query_text = read_ap.process_text(query_text)
        # print(query_text)
        sims = rank(query_text, model)
        overall_ser[qid] = dict(sims)
    print(list(overall_ser.keys())[0])
    
    return qrels, overall_ser, model, docs_by_id

if __name__ == "__main__":
    qrels, overall_ser, model, docs_by_id = main()
    evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'map', 'ndcg'})
    metrics = evaluator.evaluate(overall_ser)
    print(evaluator)
    MAP = {k: v['map'] for k, v in metrics.items()}
    all_mean_map = np.array([MAP[k] for k in MAP]).mean()
    NDCG = {k: v['ndcg'] for k, v in metrics.items()}
    all_mean_ndcg = np.array([NDCG[k] for k in NDCG]).mean()


    print(all_mean_map)
    print(all_mean_ndcg)

    print(metrics)

    # dump this to JSON
    # *Not* Optional - This is submitted in the assignment!
    with open("Doc2vec_default.json", "w") as writer:
        json.dump(metrics, writer, indent=1)
    pickle.dump(model, open( "d2v_default.p", "wb"))
    model = pickle.load(open( "d2v_default.p", "rb" ))
    query = 'Donald John Trump (born June 14, 1946) is the 45th and current president of the United States. Before entering politics, he was a businessman and television personality. Trump was born and raised in Queens, a borough of New York City, and received a bachelors degree in economics from the Wharton School. He took charge of his familys real-estate business in 1971, renamed it The Trump Organization, and expanded its operations from Queens and Brooklyn into Manhattan. The company built or renovated skyscrapers, hotels, casinos, and golf courses. Trump later started various side ventures, mostly by licensing his name.'
    print_similar(query, model, docs_by_id)
    MAP_val = {k: v['map'] for k, v in metrics.items()}
    MAP_val_int = {int(k): v for k,v in MAP_val.items()}
    MAP_val_int_test = {k: v for k,v in MAP_val_int.items() if k>= 76 and k<=100}
    val_mean = np.array([MAP_val_int_test[k] for k in MAP_val_int_test]).mean()

    NDCG_val = {k: v['ndcg'] for k, v in metrics.items()}
    NDCG_val_int = {int(k): v for k,v in NDCG_val.items()}
    NDCG_val_int_test = {k: v for k,v in NDCG_val_int.items() if k >= 76 and k<=100}
    val_mean_NDCG = np.array([NDCG_val_int_test[k] for k in NDCG_val_int_test]).mean()


    print(val_mean)
    print(val_mean_NDCG)





[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Docs already processed. Loading from disk


  0%|          | 0/149 [00:00<?, ?it/s]

Running Doc2Vec Benchmark


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 149/149 [01:13<00:00,  1.99it/s]


101
0.0034890584068787674
0.2736952551734986
{'101': {'map': 0.0011062078900552306, 'ndcg': 0.23586679340941702}, '102': {'map': 0.00012899511791429207, 'ndcg': 0.17540198977758076}, '103': {'map': 0.00038016982073334514, 'ndcg': 0.25423335310353923}, '104': {'map': 0.00023131608352207727, 'ndcg': 0.2375319423709823}, '105': {'map': 0.00012664859285102425, 'ndcg': 0.20123565033980972}, '106': {'map': 0.0002709237312863335, 'ndcg': 0.23846820626249357}, '107': {'map': 0.0013540809927411654, 'ndcg': 0.2486551732001191}, '108': {'map': 0.0007945837695974874, 'ndcg': 0.30364837688363094}, '109': {'map': 0.00011396118192769241, 'ndcg': 0.13364131903772303}, '110': {'map': 0.029145202340427317, 'ndcg': 0.550222743619465}, '111': {'map': 0.0005386011470954008, 'ndcg': 0.30300835667528636}, '112': {'map': 6.384878335865263e-05, 'ndcg': 0.14457563238087656}, '113': {'map': 0.0011514105314274102, 'ndcg': 0.24694642128877772}, '114': {'map': 0.0006058301006662802, 'ndcg': 0.31331985978106436}, '1

In [0]:
import os
import gensim
import csv
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
# import trec
import pytrec_eval
import json


import read_ap
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pickle
def rank(query, model):
      inferred_vector = model.infer_vector(query, 10000)
      sims = model.docvecs.most_similar([inferred_vector], topn = len(model.docvecs))
      return sims
def print_similar(query, model, traincorpus):
  query = read_ap.process_text(query)
  sims = rank(query, model)
  # print('Document ({}): «{}»\n'.format(doc_id, ' '.join(traincorpus[doc_id].words)))
  print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
  for label, index in [('1', 0), ('2', 1), ('3', 2), ('4', 3), ('5', 4), ('6', 5), ('7', 6), ('8', 7), ('9', 8), ('10', 9)]:
      print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(traincorpus[int(sims[index][0])])))

def main():

    qrels, queries = read_ap.read_qrels()
    docs_by_id = read_ap.get_processed_docs()

    traincorpus = [TaggedDocument(j, [i]) for i, j in docs_by_id.items()]

    for i in [200,300,400,500]:
      model = gensim.models.doc2vec.Doc2Vec(min_count=50, epochs=5)
      model.build_vocab(traincorpus)
      model.train(traincorpus, total_examples=model.corpus_count, epochs=model.epochs)
      # model = pickle.load(open( "d2v_default.p", "rb" ))


    overall_ser = {}

    print("Running Doc2Vec Benchmark")
    # collect results
    for qid in tqdm(qrels): 
        query_text = queries[qid]
        query_text = read_ap.process_text(query_text)
        # print(query_text)
        sims = rank(query_text, model)
        overall_ser[qid] = dict(sims)
    print(list(overall_ser.keys())[0])
    
    return qrels, overall_ser, model, traincorpus

if __name__ == "__main__":
    qrels, overall_ser, model, traincorpus = main()
    evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'map', 'ndcg'})
    metrics = evaluator.evaluate(overall_ser)
    print(evaluator)
    MAP = {k: v['map'] for k, v in metrics.items()}
    all_mean_map = np.array([MAP[k] for k in MAP]).mean()
    NDCG = {k: v['ndcg'] for k, v in metrics.items()}
    all_mean_ndcg = np.array([NDCG[k] for k in NDCG]).mean()


    print(all_mean_map)
    print(all_mean_ndcg)

    print(metrics)

    # dump this to JSON
    # *Not* Optional - This is submitted in the assignment!
    with open("Doc2vec_default.json", "w") as writer:
        json.dump(metrics, writer, indent=1)
    pickle.dump(model, open( "d2v_default.p", "wb"))
    model = pickle.load(open( "d2v_default.p", "rb" ))
    query = 'Donald John Trump (born June 14, 1946) is the 45th and current president of the United States. Before entering politics, he was a businessman and television personality. Trump was born and raised in Queens, a borough of New York City, and received a bachelors degree in economics from the Wharton School. He took charge of his familys real-estate business in 1971, renamed it The Trump Organization, and expanded its operations from Queens and Brooklyn into Manhattan. The company built or renovated skyscrapers, hotels, casinos, and golf courses. Trump later started various side ventures, mostly by licensing his name.'
    print_similar(query, model, traincorpus)
    MAP_val = {k: v['map'] for k, v in metrics.items() if 67<k<100}
    val_mean = np.array([MAP_val[k] for k in MAP_val]).mean()
    NDCG_val = {k: v['ndcg'] for k, v in metrics.items() if 67<k<100}
    val_mean_NDCG = np.array([NDCG_val[k] for k in NDCG_val]).mean()

    print(val_mean)
    print(val_mean_NDCG)



